In [20]:
import os
import json
from collections import defaultdict
import re
import string

In [10]:
import jieba
# 設定繁體字典
jieba.set_dictionary('dict.txt.big')
with open('wikizh_terms_short.txt', 'r') as fin:
    for line in fin.readlines():
        line = line.strip()
        jieba.add_word(line)

with open('add_word.txt', 'r') as fin: #movie titles
    for line in fin.readlines():
        jieba.add_word(line.strip().lower())

In [3]:
def strQ2B(s):
    # 全形轉半形
    new_string = ""
    for char in s:
        inside_code=ord(char) # 轉unicode
        if inside_code == 12288:                              #全形空格直接轉換            
            inside_code = 32 
        elif (inside_code >= 65281 and inside_code <= 65374): # 轉換其他全形字
            inside_code -= 65248
        new_string += chr(inside_code)
    return new_string
print(strQ2B('ＨＩ，ＯＫＯＫ'))

HI,OKOK


In [32]:
stop_words_file = 'stop_words.txt'
with open(stop_words_file) as fin:
    doc = fin.read()
stop_words = set(doc.split('\n'))

print(stop_words)

def term_extracter(content):
    content = strQ2B(content.lower())
    content = re.sub(r"[,　.;@#?!&$%^*()-=+`~{}\[\]\\\/><。| ]+\ *", ",", content)


    seg_list = jieba.lcut(content, cut_all = False) 
    seg_list = [t for t in seg_list if t not in stop_words]

    return seg_list

{'', '和', '%', '﹃', 'be', '也', '—', '﹜', 'ㄓ', '︴', '（', '才', '︰', '一', '〉', 'ㄅ', '､', '此', '只', 'is', 'ㄇ', '把', 'ㄊ', '/', '｜', '‵', '到', '[', '〝', 'ㄘ', '′', '因為', 'ㄨ', 'ˊ', '、', '那', '被', '都', '以', '﹚', '她', '什麼', 'ㄣ', 'which', 'ㄑ', '兩', 'ㄛ', '所以', '@', '是', '<', '﹕', '︽', 'ㄙ', 'are', '︶', '#', 'from', 'ㄞ', 'if', '就', '六', '？', '八', '如果', '〔', 'ㄤ', '£', '﹗', '{', '「', '〞', 'ㄡ', '在', '˙', '‘', 'ˉ', 'can', '￥', 'we', 'ㄧ', '}', 'ㄔ', 'ㄕ', '《', 'by', '說', 'it', '要', '!', '〕', '～', 'in', '將', '二', '九', '︱', '個', 'the', ',', '﹖', '能', '！', '﹒', 'ㄜ', '﹂', '已', 'ㄝ', 'ㄏ', '可', '時', '；', '又', 'ㄖ', 'ㄦ', '與', 'all', '￠', '為', '七', '各', '於', '等', '︾', '［', '三', '五', '）', '―', '﹁', '﹐', '︻', 'ㄈ', '則', '》', 'have', '會', 'this', '【', '呢', 'ㄠ', 'for', '後', ')', '這', '︸', '’', '^', '︷', '由', '讓', '﹔', 'or', 'you', 'ㄟ', '﹀', '﹄', '最', '大', '或', ' ', '做', 'ㄋ', '"', '｝', 'that', ']', '﹞', 'an', '四', '￡', '十', 'then', '」', '種', '並', '的', '】', '所', '對', '⊙', '年', '地', '再', '』', '得', '_', 'and', '>', 'ˋ', '著',

In [4]:
reviews_dir = 'yahoo_movie_review/'
directory = os.fsencode(reviews_dir)

review_paths = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".json"): 
        review_paths.append(os.path.join(reviews_dir, filename))
        

In [11]:
movie_list = []
for rpath in review_paths:
    try:
        with open(rpath, 'r') as fin:
            movie = json.load(fin, encoding='utf-8')
    except:
        continue
    movie_list.append(movie)
print(len(movie_list))
print(movie_list[0])

4550
{'length': '1時51分', 'director': '馬克威廉斯(Mark Williams)', 'm_id': 6827, 'c_title': '愛在回家時', 'review': [{'time': '2017-05-27', 'title': '《愛在回家時》回家吧，回到最初的美好。完整觀後推薦心得', 'content': '《愛在回家時》完整觀後推薦心得：http://0rz.tw/IUhKW (痞客邦)', 'rate': 5, 'author': '迪兒'}], 'date': '2017-05-26', 'img': 'https://s.yimg.com/vu/movies/fp/mpost/68/27/6827.jpg', 'e_title': 'A Family Man', 'cast': '《全面攻佔系列》傑瑞德巴特勒(Gerard Butler)、《捍衛任務》威廉達佛(Willem Dafoe)、《單身啪啪啪》愛莉森布里(Alison Brie)、《海邊的曼徹斯特》珂茜莫爾(Gretchen Mol)、《蜘蛛人2》艾爾菲摩里納(Alfred Molina)', 'category': '劇情、溫馨/家庭'}


In [18]:
mid_to_reviews = defaultdict(list)
for movie in movie_list:
    if len(movie['review']) > 149: #評論多者才比較需要分群
        m_id = movie['m_id']
        for rev in movie['review']:
            mid_to_reviews[m_id].append(rev['content'])

print(len(mid_to_reviews))
print(list(mid_to_reviews.values())[1])

433
['看完之後\n有著淡淡的哀傷\n咀嚼了一些影評和網友心得之後\n又意猶未盡的看了二遍\n這是另一種哀傷的「小倩」鬼故事\n值得一看\n&我不是工讀生', '其實沒有什麼很嚇人橋段..\r\n\r\n別懷疑跟鬼做就是這樣..呈現事實卻被說演技差? \r\n\r\n劇情普普..主要營造恐怖氣氛..這點做的不錯!!', '超讚', '結局讓人意外', '被嚇到\r\n走出電影院\r\n還在恍神', '第一次被鬼片嚇到想罵髒話.............', '不錯喔', '很久沒看過極可怕恐怖片的人一定要看！！', '以國片來說，這部片真的很不錯，看完整部片可以感覺這部劇情鋪陳得很好也很細心，要看到最後才會有恍然大悟的感覺，對國片沒信心的人很值得一看喔！', '演員生澀，演計有待加強，最多到偶像劇等級的演技\r\n劇情偶還好而已，你們到底是找了多少的工讀生來灌水?', '非常難看....評價太好，結果超失望！預告都演完了:(\n前面拖超長.....\n妝也還有很多改善空間', '國片鬼片中算是很棒的了.局情也不錯.推推推', '新竹一天只剩下一場\r\n好加在我今天趕去看了!!\r\n真的沒有讓我失望~', '還可以', '第一次被鬼片嚇的這麼徹底=-=', 'Good', '這部的題材很棒!\r\n台灣很多題材可以拍，對於身在台灣的我們，一定更有感覺!!!', '比港片鬼夜恐怖1000倍', '台灣很少能把恐怖片拍得這麼可怕~\r\n真的令我眼睛為之一亮(個人滿喜歡的,比少女時代更吸引我)\r\n劇情很流暢 把一開始不相關的東西串再一起的方式很棒!\r\n只有最後結局有點沒說完的感覺\r\n前世今生的地方說的不夠清楚~\r\n所以三顆星+一顆星國片加油!!', '題材新鮮~故事連貫性好~讓人有印象的內容!!!', '沒有想像中好看，重點都在預告裡了。', '好看', '起承轉合一步有頭尾的正宗鬼片還融合台灣習俗，好看沒話說', '普通', '謝欣穎好正', '就是SO SO而已', '超極好看', '恐怖其實還OK,但是很喜歡這種有邏輯性的電影\r\n劇情相當不錯,值得大推', '雖然被嚇死 但真的值得一看超讚', '突破台灣鬼片的困境', '沒想到臺灣鬼片能拍的這麼好\n轉折點跟結局有點弱就是了', '看完之後毛毛的...看完之後回家也一直亂想= =', '鈴

In [38]:
def get_review_sents(review_lst, tokenize=True):
    review_sent_lst = []
    min_sent_len = 1
    for review in review_lst:
        sentences = re.split('[{}{}]+'.format(string.punctuation, '\n\r'), strQ2B(review))
        sentences = [sent.strip() for sent in sentences if len(sent) > min_sent_len]
        if tokenize:
            sentences = [' '.join(term_extracter(sent)) for sent in sentences]
        review_sent_lst.append(sentences)
    return review_sent_lst

In [39]:
mid_to_review_in_sents = dict()
for mid, review_lst in mid_to_reviews.items():
    mid_to_review_in_sents[mid] = get_review_sents(review_lst, tokenize=False)

mid_to_review_in_sents_token = dict()
for mid, review_lst in mid_to_reviews.items():
    mid_to_review_in_sents_token[mid] = get_review_sents(review_lst, tokenize=True)


In [40]:
# mid_to_review_in_sents[4116]
mid_to_review_in_sents_token[4116]

[['感人 好笑 xdd ok 梗 好 經典 hahaha'],
 ['還 不錯', '劇情 環環相扣 喜歡 黑洞 頻率 回到未來 一定 喜歡 這類 劇情 取向 單純 喜歡 看 打打殺殺 電影 特效 推薦'],
 ['好片 解釋'],
 ['劇情 環環相扣 很 精彩'],
 ['簡單 來說 非常 好看'],
 ['可是 穿越時空 地方 可以 描寫 細膩', '否則 觀眾 一直 找出 縫隙 矛盾 點', '主角 們 帥 好笑'],
 ['很 想 解說 沒 想像中 好看 該 多 好看', '此人 想象 難道 另外 編劇', '太有'],
 ['看到 想 睡覺', '真的 好看 連看 沒 看 完'],
 ['覺得 好看 看 一次',
  '想 第一次 看 節奏 有點 急',
  '第二次 看 懂 囉',
  '超多 笑點 超多 梗',
  '後面 超 感人',
  'ok 不想 j 知道 事 就是 爸爸 一個 笑點 j 克萊斯勒 大樓 跳 下 來時 時間 一度 回到 華爾街 崩盤 旁邊 一些 正在 跳樓 囉',
  '發生 事情 拿來 串連 劇情 編劇 真的 很 不錯',
  '可惜 節奏 太快 一般 觀眾 或者 懂 歷史 懂 棒球 來不及 看不懂',
  '這集 真的 太 精典',
  '期待 第四集'],
 ['有些 劇情 很 有意思'],
 ['顛覆 第 集 印象', '劇情 整個 老梗 爆'],
 ['笑 淚', '劇情 緊湊 值得一看'],
 ['完美 好看 無聊'],
 ['翻閱 大多數 網友 短評',
  '絕大多數 沒前 幾集 好看 有點 小 失落 好好笑 劇情 老梗 後面 很 感人 很 推薦 等等 好像 沒 怎麼 這部 片 帶來 涵義 小弟 也許 看到 多',
  '想 我知道 這部 片想 表達 一件 小小的 事 可以 帶來 很大 效應',
  '即使 是給 小費 一個 小小的 舉動',
  '可以 避免 隕石 撞 地球',
  '拯救 世界',
  '相信 看 這部 片 大家 應該 看到',
  '不是 洩漏 劇情',
  '至於 衛星 跑 那邊 隕石 撞',
  '猜想 每個 mib 一個 衛星 鎖定 這樣 才能 知道 mib 探員 哪',
  '走 過去 衛星 可能 需要 配合 改變 軌道 並非 不無 可能 勸 多 一些 善事 或是 好事',
  

In [41]:
with open('data/mid_to_reviews_in_sents.json','w') as fout:
    json.dump(mid_to_review_in_sents, fout)

In [43]:
with open('data/mid_to_reviews_in_sents_token.json', 'w') as fout:
    json.dump(mid_to_review_in_sents_token, fout)

In [7]:
titles = []
for m in movie_list:
    titles.append(m['c_title'])

print(titles[:100])

['愛在回家時', '1911-辛亥革命', '天才無限家', '為愛遠離', '尪某愛計較', '媽媽，晚餐吃什麼？', '愛人怪物', '全家玩到趴', '悲慘世界', '喬治羅密歐之活屍日記', '蜂蜜之夏', '竊盜城', '愛，不怕', '尋找天堂的3個人', '蠟筆小新：我的超時空新娘', '大河戀20週年經典數位修復版', '沈默茱麗葉', '我的外籍老師', '蔡李佛', '男生女生騙', '中央情爆員', '火影忍者疾風傳', '一路向西', '千王新世紀', '美麗的約定', '拼圖女王', '伴我走天涯', '屍憶', '分居風暴', '愛的成人式', '血色入侵', '啟動原始碼', '破風', '神偷奶爸2', '5月天諾亞方舟3D電影', '喜羊羊與灰太狼4：開心闖龍年', '3人擠不擠', '翻聲吧！保羅', '惡夢探偵', '盲流感', '鬼片', '決戰異世界：鬼哭狼嚎', '逃離邊境', '換命法則', '禁愛風暴', '憂傷思念曲', '孤兒怨', '暮光之城：無懼的愛', '雲端求愛記', '哈拉夜未眠', '二姊妹的情人', '天籟美聲', '臨門湊一腳', '我想念我自己', '魔鬼剋星', '推理要在晚餐後', '走出五月', '花落花開', '惡作劇之刎', '亡命殺鎮', '歌劇魅影 舞台版', '梭哈人生', '天之茶助', '明明', '非誠勿擾2', '死神', '歡迎光臨貝拉的奇幻花園', '他其實沒那麼喜歡妳', '鬼訊號2：靈異透視', '3D王牌小間諜', '三槍拍案驚奇', '金錢怪獸', '決勝機密', '黑勢力', 'Free！劇場版《High Speed！─Free！Starting Days─》', '心靈暗湧', '髮膠明星夢', '百萬殺人遊戲', '小王子', '她其實沒那麼壞', '家有兩個爸', '蘇乞兒', '無處可逃', '高達與米耶維勒雙回顧展', '久美子的奇異旅程', '勇者物語', '海底總動員2：多莉去哪兒？', '狐仙麗莎煞煞煞', '神奇俠侶', '鯰魚很驚奇', '來約會8！', '魔法阿爸', '冷光線索', '雪地迷蹤', '一個人的收藏', '愛愛上雲端', '經典華語歌唱電影回顧展', '惡夜殺機', '總統', '時空幻境之宵星傳奇']
